In [1]:
%run train.py

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

fmt = '%(asctime)s:%(msecs)d - %(levelname)s'
fmt += ' - {%(filename)s:%(lineno)d} %(message)s'

logging.basicConfig(level=logging.DEBUG,
                    format=fmt,
                    datefmt='%m-%d %H:%M:%S',
                    filename='widgets.log',
                    filemode='w')
logging.info("Creating widgets.log file")

In [11]:
# interrogate about the job
import requests
job_nb = 2
c = requests.get(f"http://10.10.77.61:8500/train?service=tls_park&job={job_nb}&timeout=10")
c.json()

{'status': {'code': 400,
  'dd_code': 1002,
  'dd_msg': 'Service Not Found',
  'msg': 'NotFound'}}

In [4]:
s.run()

In [3]:
    
s = Segmentation(
    'tls_park', 
    training_repo='/data1/sat/dataset_tls_park_444/train.txt',
    testing_repo='/data1/sat/dataset_tls_park_444/test.txt',
    host='10.10.77.61',
    port=8500,
    #port=8400,
    img_height=480,
    img_width=480,
    model_repo='/data1/sat/xolive/models/model_tls_park_debug',
    nclasses=2,
    #template='deeplab_vgg16',
    iterations=15000,
    test_interval=500,
    snapshot_interval=1000,
    batch_size=5,
    test_batch_size=1,
    noise_prob=0.001,
    class_weights=[0.1, 1.0],
    model_postfix='test_park_xo3',
    weights='/data1/segmentation/vgg16_init_deeplab.caffemodel',
    gpuid=2
)#.run()
s

In [19]:
c = Classification('xo_bbc_5_classes', 
        training_repo='/data1/xolive/bbc_sounds/sounds_dataset_1.0/train/',
        testing_repo='/data1/xolive/bbc_sounds/sounds_dataset_1.0/test/',
                   host='10.10.77.61',
        port=8500,
        model_repo='/data1/bbc_sounds/bbc_5_classes',
        layers='["1CR32", "1CR64", "1CR128", "1024"]',
        template='convnet',
        gpuid=1,
                   img_width=257,
                   img_height=257
        )#.run()
c
# TODO unchanged data

In [ ]:
c.clear()

In [1]:
# interrogate about the job
import requests
job_nb = 1
c = requests.get(f"http://10.10.77.61:8500/train?service=xo_bbc_5_classes&job={job_nb}&timeout=10")
c.json()

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [18]:
c.clear()

AttributeError: 'Response' object has no attribute 'clear'

In [ ]:
c.del_request

In [ ]:
c= requests.delete("http://10.10.77.61:8500/services/xo_bbc_5_classes?clear=full")

In [3]:
%run train.py

In [7]:
c = Classification('kaggle', 
                    training_repo='/data1/kaggle/dogs_cats/train/',
                    #testing_repo='/data1/xolive/bbc_sounds/sounds_dataset_1.0/test/',
                    host='10.10.77.61',
                    port=8500,
                    model_repo='/data1/xolive/models/kaggle',
                    template='convnet',
                    gpuid=1,
                   layers=['1CR32','1CR64','1CR128','1024'],
                   img_width=150,
                   img_height=150,
                   mirror=True,
                   rotate=True,
                   tsplit=0.2
                   
        )#.run()
c
# TODO unchanged data

In [5]:
c.status()

AttributeError: 'Classification' object has no attribute 'status'

In [8]:
# interrogate about the job
import requests
job_nb = 1
c = requests.get(f"http://10.10.77.61:8500/train?service=kaggle&job={job_nb}&timeout=10")
c.json()

{'body': {'measure': {}},
 'head': {'job': 1, 'method': '/train', 'status': 'running', 'time': 28.0},
 'status': {'code': 200, 'msg': 'OK'}}